# Dashboard for Time series Forecasting and Visualization

#### by João Vidigal

_____

## Objective: 
 Explore different stocks and forecast the value for the enxt days in a easy to use dashboard.
### Dashboard architecture 
Three major components compose this dashboard architecture:

* 1. [Data Collection](#DataCollection) done by retrieving Yahoo financial data and transform it to incorporate it on later model.

* 2. [Forecast Model](#Model) using Facebook forecast library. This model is based on the data collected before.

* 3. [Dashboard Layout](#Dashboard) built using panel library. The dashboard enables interactive visualization. Also, enables going through new stocks without changing code. 


## Imports

In [7]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import datetime
from yahoofinancials import YahooFinancials

from typing import Union, Dict, List, Callable
import statsmodels
from statsmodels.nonparametric.smoothers_lowess import lowess
import statsmodels.api as sm

import fbprophet
from fbprophet import Prophet

import bokeh
from bokeh.io import output_notebook, show
from bokeh.models import CustomJS, ColumnDataSource, Slider, Label, Div, HoverTool, Band, Span, BoxAnnotation
from bokeh.plotting import figure, show
from bokeh.palettes import Spectral11
from bokeh.palettes import Category10
from bokeh.io import output_notebook

import hvplot.pandas
import panel as pn
import datetime as dt
import param


<a id="DataCollection"></a>
## Data Collection
Analysing and manipulating Yahoo financial data uisng the [yahoofinancials library](https://github.com/JECSand/yahoofinancials).  

Based on specific dates it will return a pandas dataframe with timeseries information concerning the values and volumes of the chosen stocks. 

In [3]:

#####################################################################################
###  Data scrapping from Yahoo Finance
#####################################################################################

today = str(datetime.date.today())

class finance():
    """ Finance class for analysis/manipulating yahoo financial data
    
        Returns the Adjusted Closing Price for a specfic stock or financial asset between selected dates
    
    Input: 
        stock: Can be a Stock tickers or a list (eg.'MSFT'or ['MSFT', 'AAPL','AMZN'] ).
        since: Date since it will return the data (eg. '2019-01-01').
        close: Date until it will return the data (eg. '2019-02-01'), by standard the date is the one from today.
        
    Output:
        Dataframe similar to the one below if .df is used after the class.
        
                open	high	low	close	adjclose	volume
    date						
    2019-01-02	154.889999	158.850006	154.229996	157.919998	156.642365	37039700
    2019-01-03	143.979996	145.720001	142.000000	142.190002	141.039642	91244100
    2019-01-04	144.529999	148.550003	143.800003	148.259995	147.060516	58607100
    2019-01-07	148.699997	148.830002	145.899994	147.929993	146.733185	54777800
    
    eg.
        finance('AAPL', '2019-01-01').df

    
    """
    
    def __init__(self, stock, since, close=today):
        self.stock = stock
        self.since = since
        self.close = close
        yahoo_financials = YahooFinancials(stock)
        today = str(datetime.date.today())
        historical_stock_prices = yahoo_financials.get_historical_price_data(since, close, 'daily')
        df = pd.DataFrame()
        
        
        datetime_object_since = datetime.datetime.strptime(since, '%Y-%m-%d')
        year_since = int(datetime_object_since.strftime('%Y'))
        month_since = int(datetime_object_since.strftime('%m'))
        day_since = int(datetime_object_since.strftime('%d'))

        datetime_object_close = datetime.datetime.strptime(close, '%Y-%m-%d')
        year_close = int(datetime_object_close.strftime('%Y'))
        month_close = int(datetime_object_close.strftime('%m'))
        day_close = int(datetime_object_close.strftime('%d'))
        
        # for single stock
        if isinstance(stock, str):
            df = pd.DataFrame.from_dict(historical_stock_prices[stock]['prices'])
            df = df.drop('date', axis=1)
            df['date'] = df['formatted_date']
            df['date'] =  pd.to_datetime(df['date'])
            self.df = df[['date','open', 'high', 'low', 'close', 'adjclose','volume']]
        
        # or for a list of stock
        else:
            for i in stock:
                df1 = pd.DataFrame.from_dict(historical_stock_prices[i]['prices'])
                df1['ticker'] = i
                df = df.append(df1, ignore_index=True)
                df = df.drop('date', axis=1)
                df['date'] = df['formatted_date']
                df['date'] =  pd.to_datetime(df['date'])
                self.df = df[['ticker','date','open', 'high', 'low', 'close', 'adjclose','volume']]
                

<a id="Model"></a>
## Forecast Model
Fbprophet library by Facebook was designed for business time Series, to be robust and easy to use. Like in other time series forecasting methods can handle trend and seasonal data. Prophet is a curve-fitting model.

RMSE was the metric chosen to evaluate the forecast. The time Series were visualized using the bokeh library to enable interactivity.

The prophet_show() enables to run the model and the prophet_predict() to forecast the next time points based on the previous model.



In [4]:
####################################################################################
###  Forecasting with Fbprophet
#####################################################################################


def rmse(y: Union[np.ndarray, float], yhat: Union[np.ndarray, float]) -> float:
    """RMSE evaluation metric"""
    result = np.sqrt(((yhat - y) ** 2))
    return result

def rmse_df(df: pd.DataFrame) -> pd.DataFrame:
    return rmse(df.y, df.yhat)


def prophet_show(df_train, date, y, cutoff_train, cutoff_eval, prophet_kwargs, title):
    ts = (df_train.query('date < @cutoff_eval')[[date, y]]
          .rename(columns={date:'ds', y:'y'})).reset_index(drop=True)
    ind_train = pd.eval('ts.ds < cutoff_train')
    ind_eval = ~ ind_train
    len_train, len_eval = ind_train.sum(), ind_eval.sum()
    ts_train = ts.loc[ind_train]
    m = Prophet(**prophet_kwargs)
    m.fit(ts_train)
    ts_hat = m.predict(ts).merge(ts[['ds', 'y']], on='ds', how='left')

    df_combined = ts_hat.assign(rmse=0, rmse_smooth=0)
    df_combined.rmse = rmse_df(df_combined)
    df_combined.loc[ind_train, 'rmse_smooth'] = lowess(df_combined.loc[ind_train, 'rmse'], range(len_train), frac=0.03, return_sorted=False)
    df_combined.loc[ind_eval, 'rmse_smooth'] = lowess(df_combined.loc[ind_eval, 'rmse'], range(len_eval), frac=0.35, return_sorted=False)
    rmse_in = df_combined.loc[ind_train].rmse.mean()
    rmse_oos = df_combined.loc[ind_eval].rmse.mean()
    
    source = ColumnDataSource(data=df_combined)
    p = figure(plot_width=950, plot_height=300, title=("{}   train / test = ..{} / ..{}"
                                                       .format(title, cutoff_train, cutoff_eval)), 
               x_axis_type='datetime', tools="pan, box_zoom, wheel_zoom,reset")
    _ = p.line(x='ds', y='yhat', source=source)
    _ = p.line(x='ds', y='yhat_lower', source=source, line_alpha=0.4)
    _ = p.line(x='ds', y='yhat_upper', source=source, line_alpha=0.4)
    _ = p.scatter(x='ds', y='y', source=source, color='black', radius=0.2, radius_dimension='y', alpha=0.4)
    _ = p.scatter(x='ds', y='y', source=source, color='black', radius=0.2, radius_dimension='y', alpha=0.4)
    
    p.add_tools(HoverTool(tooltips=[('Date','@ds{%F}'),('yhat', '@yhat'),('y', '@y')], mode='mouse',  formatters={'ds':'datetime'}))
    p.add_layout(BoxAnnotation(left=ts_train.ds.iloc[-1], right=ts.ds.iloc[-1]))  
    
    p2 = figure(plot_width=950, plot_height=200, title="RMSE Train / Test = {:.3f} / {:.3f}".format(rmse_in, rmse_oos), x_axis_type='datetime', tools="",
                x_range=p.x_range)
    sm1 = p2.line(x='ds', y='rmse_smooth', source=source, color='green')
    p2.add_tools(HoverTool(tooltips=[('rmse', '@rmse')], renderers=[sm1], mode='vline', line_policy='interp'))
    p2.add_layout(BoxAnnotation(left=ts_train.ds.iloc[-1], right=ts.ds.iloc[-1]))
    p2.yaxis[0].ticker.desired_num_ticks = 2
    #bokeh.io.show(bokeh.layouts.column(p, p2))
    bokeh_pane = pn.pane.Bokeh(bokeh.layouts.column(p, p2))

    return  m , bokeh_pane

def prophet_predict(model, periods, freq='d'):
    future = model.make_future_dataframe(periods, freq)
    forecast = model.predict(future)
    forecast1 = forecast[(forecast['ds'] > today)]
    return forecast1

<a id="Dashboard"></a>
## Dashboard Layout
Panel is a new open-source Python library that lets you create custom interactive web apps and dashboards.

In [5]:
####################################################################################
###  Dashboard
#####################################################################################


class PredictStock(param.Parameterized):
    """
    Dashboard 
    """
    
    today = str(datetime.date.today())
    ticker = param.String(default='AAPL')
    time_start = param.String(default='2013-01-01')
    window_num = param.Number(default=30)
    cutoff_train_dt = param.String(default='2019-08-20')
    cutoff_eval_dt = param.String(default=today)
    periods = param.Number(default=30)
 
    @param.depends('ticker')
    def header(self):
        return pn.Row('## Time series analysis of ' +  self.ticker )
    
    @param.depends('ticker','time_start')
    def data(self):
        df = finance( self.ticker, self.time_start).df
        return df

    @param.depends('data','window_num')
    def graph(self):
        
        df = self.data()
        df['date'] =  pd.to_datetime(df['date'])
        df = df.set_index('date')
        df['rolling_mean'] = df['adjclose'].rolling(window=self.window_num).mean()
        return df[['adjclose','rolling_mean']].hvplot() 
    
    @param.depends('data')
    def table(self):
        return self.data().describe()
    
    @param.depends('data','cutoff_train_dt','cutoff_eval_dt')
    def prophet(self):
        model , bokeh_pane = prophet_show(self.data(), 'date', 'adjclose',cutoff_train=self.cutoff_train_dt, cutoff_eval= self.cutoff_eval_dt,
             prophet_kwargs={'yearly_seasonality':True, 'weekly_seasonality':True,
                            'uncertainty_samples':500}, title='Time series forecast by Prophet')
        return  model, bokeh_pane
    
    @param.depends('prophet')
    def graph_prophet(self):
        model, bokeh_pane = self.prophet()
        return  bokeh_pane

    @param.depends('prophet','periods')
    def predict(self):
        model, bokeh_pane = self.prophet()
        df = prophet_predict(model, self.periods)
        df['date'] = df['ds']
        df = df.set_index('date')
        df = df[['yhat_lower', 'yhat' ,'yhat_upper']]
        return  df
    
    def panel(self):
        return  pn.Row(self.param,
                      pn.Column(self.header, pn.Row(self.graph, self.table), 
                      pn.Row('##Forecast and prediction'),pn.Row(self.graph_prophet,'        ', self.predict)), 
                         min_height=1000)
    


In [6]:
reactive = PredictStock()
reactive.panel().servable()

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:bokeh.server.server:Starting Bokeh server version 1.3.4 (running on Tornado 6.0.3)


INFO:tornado.access:200 GET / (::1) 468.82ms
INFO:tornado.access:200 GET /static/js/bokeh-tables.min.js?v=5f778b8a005d8538b5b13998ec45fc16 (::1) 4.67ms
INFO:tornado.access:200 GET /static/js/bokeh-gl.min.js?v=be19384f76795da42f51580e7b5fd473 (::1) 8.18ms
INFO:tornado.access:200 GET /static/js/bokeh.min.js?v=547e7d2591695b654def5914bdd697fa (::1) 12.90ms
INFO:tornado.access:200 GET /static/js/bokeh-widgets.min.js?v=423bf6bb32b8def8b6c9df74817506e4 (::1) 15.48ms
INFO:tornado.access:101 GET /ws?bokeh-protocol-version=1.0&bokeh-session-id=10HB8niVS5hNIVaC2lgDo9g4n9ia0HCmkkyJI1DtadDg (::1) 0.76ms
INFO:bokeh.server.views.ws:WebSocket connection opened
INFO:bokeh.server.views.ws:ServerConnection created
INFO:bokeh.server.views.ws:WebSocket connection closed: code=1001, reason=None


Based on tutorials from:

https://www.kaggle.com/myster/eda-prophet-winning-solution-3-0  
https://holoviz.org/  
https://github.com/JECSand/yahoofinancials 